In [3]:
import os
import pickle
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [4]:
# init dataloading (load the last trial)
file_storage_found = False
idx_file_storage = 1
while not file_storage_found:
    file_storage_path = './results/experiment_' + str(idx_file_storage) + '.pkl'
    if os.path.isfile(file_storage_path):
        idx_file_storage += 1
    else:
        file_storage_path = './results/experiment_' + str(idx_file_storage-1) + '.pkl'
        file_storage_found = True

# put id here if NOT last should be loaded
# file_storage_path = './results/experiment_' + str(idx_file_storage) + '.pkl'
file_storage_path = './results/experiment_50_100.pkl'

with open(file_storage_path, 'rb') as f:
    data = pickle.load(f)

# seperate best individual and data
best_individual = data[1::2]
data = data[::2]
# select only data from best individual
all_data = []
data_best_individual = []
for counter, individual in enumerate(data):
    all_data = np.append(all_data, individual)
    data_best_individual.append(individual[best_individual[counter]])
    
df_plot = pd.DataFrame.from_records(all_data)

## Select the first 600 trials only

In [5]:
# df_plot = df.iloc[:600,6:-8]
df_plot.sort_values(by='fitness', ascending=False, inplace=True)
df_plot.head()

,individual,a,A1,A2,fitness
150,1,-1235.386034,1.190189,0.313551,93.352221
81,32,-1235.386034,1.190189,0.313551,91.065705
100,1,-1235.386034,1.190189,0.313551,90.753777
200,1,-1235.386034,1.190189,0.313551,89.860920
84,35,-397.073337,-9.126374,5.700049,85.946085


In [6]:
axis_label_size = 18
axis_ticks_size = 14
dimension_label_size = axis_label_size*1
dimension_ticks_size = axis_ticks_size*1
cbar_label_size = 16
cbar_ticks_size = 14
legend_title_size = 'x-large'
legend_entry_size = 14

In [32]:
min_acc = np.round(np.nanmin(df_plot["fitness"]),-1)
min_bar = (int(str(min_acc)[0])-1)*10**(len(str(int(min_acc)))-1)

max_acc = np.round(np.nanmax(df_plot["fitness"]),-1)
max_bar = (int(str(max_acc)[0])+1)*10**(len(str(int(min_acc)))-1)

In [50]:
# fix number of ticks and set precision
tickvals_a = np.linspace(np.nanmin(df_plot['a']), np.nanmax(df_plot['a']), 10)
tickvals_a = tickvals_a.round(0)

tickvals_A1 = np.linspace(np.nanmin(df_plot['A1'])*0.95, np.nanmax(df_plot['A1'])*1.05, 10)
tickvals_A1 = tickvals_A1.round(2)

tickvals_A2 = np.linspace(np.nanmin(df_plot['A2'])*0.95, np.nanmax(df_plot['A2'])*1.05, 10)
tickvals_A2 = tickvals_A2.round(2)

## Plot all together

In [52]:

fig = go.Figure(layout=dict(width=1000, height=500, margin=go.layout.Margin(l=50, r=20, b=20, t=50, pad=0)),
                data=go.Parcoords(
    line=dict(color=df_plot['fitness'],
                colorscale=px.colors.diverging.Portland,
                showscale=True,
                cmin=0,
                cmax=100,
                colorbar={"title": dict(text="Accuracy (%)", side="right", font=dict(size=cbar_label_size)),
                          "tickvals": np.arange(0, 100+1, 10),
                          "tickfont": dict(size=cbar_ticks_size)}),
    dimensions=list([
        dict(tickvals=tickvals_a,
            label="a", values=df_plot['a']),
        dict(tickvals=tickvals_A1,
            label='A1', values=df_plot['A1']),
        dict(tickvals=tickvals_A2,
            label='A2', values=df_plot['A2']),
        dict(range=[0, 100],
             tickvals=np.arange(0, 100+1, 20),
             label='Accuracy (%)', values=df_plot['fitness'])
    ]),
    labelfont=dict(size=dimension_label_size),
    tickfont=dict(size=dimension_ticks_size)
)
)
fig.show()


## Split into 10 (in color) and 590 (in grayscale)

In [85]:
df_plot1 = df_plot.iloc[:10, :]
df_plot2 = df_plot.iloc[10:, :]

In [86]:
df_plot1.head()
print(min(df_plot1["default"]))
print(max(df_plot1["default"]))

72.23462301587301
75.29761904761905


In [87]:
df_plot2.head()

,parameters.scale,parameters.time_bin_size,parameters.nb_input_copies,parameters.tau_mem,parameters.tau_ratio,parameters.fwd_weight_scale,parameters.weight_scale_factor,parameters.reg_spikes,parameters.reg_neurons,default
489,10.0,5.0,8,0.06,10,3.5,0.050,0.0025,0.0,72.222222
488,10.0,5.0,2,0.05,10,3.5,0.050,0.0030,0.0,72.135417
522,10.0,5.0,2,0.05,10,3.5,0.045,0.0030,0.0,71.701389
457,10.0,5.0,8,0.05,10,3.5,0.045,0.0035,0.0,71.664187
181,35.0,7.0,4,0.06,2,2.0,0.030,0.0010,0.0,71.626984


In [88]:
min_top_ten = min(df_plot1["default"])*1E-2
max_top_ten = max(df_plot1["default"])*1E-2
min_all = min(df_plot["default"])*1E-2
max_all = max(df_plot["default"])*1E-2
print(min_all, max_all, min_top_ten, (min_top_ten+max_top_ten)/2, max_top_ten)
print((min_top_ten/max_all), (((min_top_ten+max_top_ten)/2)/max_all), (max_top_ten/max_all))
fig = go.Figure(layout=dict(width=2000, height=500, margin=go.layout.Margin(l=40, r=20, b=20, t=50, pad=0)),
                data=go.Parcoords(
    line=dict(color=df_plot['default'],
                colorscale=[[0, 'white'], [0.9,'lightgrey'], [0.91,'blue'], [1, 'red']],
                showscale=False,
                cmin=min_all*1E2,
                cmax=max_all*1E2,
                colorbar={"title": dict(text="Accuracy (%)", side="right", font=dict(size=cbar_label_size)),
                          "tickvals": np.arange(10, 90+1, 10),
                          "tickfont": dict(size=cbar_ticks_size)}),
    dimensions=list([
        dict(tickvals=np.arange(np.nanmin(df_plot['parameters.scale']),
                                np.nanmax(
            df_plot['parameters.scale'])*1.05,
            15),
            label="scale", values=df_plot['parameters.scale']),
        dict(tickvals=np.arange(np.nanmin(df_plot['parameters.time_bin_size']),
                                np.nanmax(
            df_plot['parameters.time_bin_size'])*1.05,
            1),
            label='time_bin_size (ms)', values=df_plot['parameters.time_bin_size']),
        dict(tickvals=[1, 2, 4, 8],
             label='nb_input_copies', values=df_plot['parameters.nb_input_copies']),
        dict(tickvals=np.arange(np.nanmin(df_plot['parameters.tau_mem'])*1000,
                                np.nanmax(
            df_plot['parameters.tau_mem'])*1000*1.05,
            10),
            label='tau_mem (ms)', values=df_plot['parameters.tau_mem']*1000),
        dict(tickvals=[1, 2, 5, 10],
             label='tau_ratio', values=df_plot['parameters.tau_ratio']),
        dict(tickvals=np.arange(np.nanmin(df_plot['parameters.fwd_weight_scale']),
                                np.nanmax(
            df_plot['parameters.fwd_weight_scale'])*1.05,
            1),
            label='fwd_weight_scale', values=df_plot['parameters.fwd_weight_scale']),
        dict(tickvals=np.arange(np.nanmin(df_plot['parameters.weight_scale_factor']),
                                np.nanmax(
            df_plot['parameters.weight_scale_factor'])*1.05,
            15e-3),
            tickformat=".0s",
            label='weight_scale_factor', values=df_plot['parameters.weight_scale_factor']),
        dict(tickvals=np.arange(np.nanmin(df_plot['parameters.reg_spikes']),
                                np.nanmax(
            df_plot['parameters.reg_spikes'])*1.05,
            1e-3),
            tickformat=".0s",
            label='reg_spikes', values=df_plot['parameters.reg_spikes']),
        dict(tickvals=np.arange(np.nanmin(df_plot['parameters.reg_neurons']),
                                np.nanmax(
            df_plot['parameters.reg_neurons'])*1.05,
            2e-6),
            tickformat=".0s",
            label='reg_neurons', values=df_plot['parameters.reg_neurons']),
        dict(range=[10, 90],
            #  constraintrange = [70, 100],
             tickvals=np.arange(10, 90+1, 20),
             label='Accuracy (%)', values=df_plot['default'])
    ]),
    labelfont=dict(size=dimension_label_size),
    tickfont=dict(size=dimension_ticks_size)
)
)
fig.show()


0.20572916666666666 0.7529761904761906 0.7223462301587301 0.7376612103174603 0.7529761904761906
0.9593214756258233 0.9796607378129116 1.0


In [89]:
image_filename = "NNI_th{}".format(threshold)

fig.write_image("./{}.pdf".format(image_filename))
fig.write_image("./{}.png".format(image_filename))

## Plot one after the other to check data selection and color settings

In [90]:
fig = go.Figure(layout=dict(width=2000, height=500, margin=go.layout.Margin(l=40, r=20, b=20, t=50, pad=0)),
                data=go.Parcoords(
    line=dict(color=df_plot1['default'],
                colorscale=px.colors.diverging.Portland,
                showscale=True,
                cmin=10,
                cmax=90,
                colorbar={"title": dict(text="Accuracy (%)", side="right", font=dict(size=cbar_label_size)),
                          "tickvals": np.arange(10, 90+1, 10),
                          "tickfont": dict(size=cbar_ticks_size)}),
    dimensions=list([
        dict(tickvals=np.arange(np.nanmin(df_plot1['parameters.scale']),
                                np.nanmax(
            df_plot['parameters.scale'])*1.05,
            15),
            label="scale", values=df_plot1['parameters.scale']),
        dict(tickvals=np.arange(np.nanmin(df_plot['parameters.time_bin_size']),
                                np.nanmax(
            df_plot['parameters.time_bin_size'])*1.05,
            1),
            label='time_bin_size (ms)', values=df_plot1['parameters.time_bin_size']),
        dict(tickvals=[1, 2, 4, 8],
             label='nb_input_copies', values=df_plot1['parameters.nb_input_copies']),
        dict(tickvals=np.arange(np.nanmin(df_plot['parameters.tau_mem'])*1000,
                                np.nanmax(
            df_plot['parameters.tau_mem'])*1000*1.05,
            10),
            label='tau_mem (ms)', values=df_plot1['parameters.tau_mem']*1000),
        dict(tickvals=[1, 2, 5, 10],
             label='tau_ratio', values=df_plot1['parameters.tau_ratio']),
        dict(tickvals=np.arange(np.nanmin(df_plot['parameters.fwd_weight_scale']),
                                np.nanmax(
            df_plot['parameters.fwd_weight_scale'])*1.05,
            1),
            label='fwd_weight_scale', values=df_plot1['parameters.fwd_weight_scale']),
        dict(tickvals=np.arange(np.nanmin(df_plot['parameters.weight_scale_factor']),
                                np.nanmax(
            df_plot['parameters.weight_scale_factor'])*1.05,
            15e-3),
            tickformat=".0s",
            label='weight_scale_factor', values=df_plot1['parameters.weight_scale_factor']),
        dict(tickvals=np.arange(np.nanmin(df_plot['parameters.reg_spikes']),
                                np.nanmax(
            df_plot['parameters.reg_spikes'])*1.05,
            1e-3),
            tickformat=".0s",
            label='reg_spikes', values=df_plot1['parameters.reg_spikes']),
        dict(tickvals=np.arange(np.nanmin(df_plot['parameters.reg_neurons']),
                                np.nanmax(
            df_plot['parameters.reg_neurons'])*1.05,
            2e-6),
            tickformat=".0s",
            label='reg_neurons', values=df_plot1['parameters.reg_neurons']),
        dict(range=[10, 90],
             tickvals=np.arange(10, 90+1, 20),
             label='Accuracy (%)', values=df_plot1['default'])
    ]),
    labelfont=dict(size=dimension_label_size),
    tickfont=dict(size=dimension_ticks_size)
)
)
fig.show()


In [91]:
fig = go.Figure(layout=dict(width=2000, height=500, margin=go.layout.Margin(l=40, r=20, b=20, t=50, pad=0)),
                data=go.Parcoords(
    line=dict(color=df_plot2['default'],
                colorscale=px.colors.sequential.Greys,
                showscale=False,
                cmin=10,
                cmax=90,
                colorbar={"title": dict(text="Accuracy (%)", side="right", font=dict(size=cbar_label_size)),
                          "tickvals": np.arange(10, 90+1, 10),
                          "tickfont": dict(size=cbar_ticks_size)}),
    dimensions=list([
        dict(tickvals=np.arange(np.nanmin(df_plot['parameters.scale']),
                                np.nanmax(
            df_plot['parameters.scale'])*1.05,
            15),
            label="scale", values=df_plot2['parameters.scale']),
        dict(tickvals=np.arange(np.nanmin(df_plot2['parameters.time_bin_size']),
                                np.nanmax(
            df_plot2['parameters.time_bin_size'])*1.05,
            1),
            label='time_bin_size (ms)', values=df_plot2['parameters.time_bin_size']),
        dict(tickvals=[1, 2, 4, 8],
             label='nb_input_copies', values=df_plot2['parameters.nb_input_copies']),
        dict(tickvals=np.arange(np.nanmin(df_plot['parameters.tau_mem'])*1000,
                                np.nanmax(
            df_plot['parameters.tau_mem'])*1000*1.05,
            10),
            label='tau_mem (ms)', values=df_plot2['parameters.tau_mem']*1000),
        dict(tickvals=[1, 2, 5, 10],
             label='tau_ratio', values=df_plot2['parameters.tau_ratio']),
        dict(tickvals=np.arange(np.nanmin(df_plot['parameters.fwd_weight_scale']),
                                np.nanmax(
            df_plot['parameters.fwd_weight_scale'])*1.05,
            1),
            label='fwd_weight_scale', values=df_plot2['parameters.fwd_weight_scale']),
        dict(tickvals=np.arange(np.nanmin(df_plot['parameters.weight_scale_factor']),
                                np.nanmax(
            df_plot['parameters.weight_scale_factor'])*1.05,
            15e-3),
            tickformat=".0s",
            label='weight_scale_factor', values=df_plot2['parameters.weight_scale_factor']),
        dict(tickvals=np.arange(np.nanmin(df_plot['parameters.reg_spikes']),
                                np.nanmax(
            df_plot['parameters.reg_spikes'])*1.05,
            1e-3),
            tickformat=".0s",
            label='reg_spikes', values=df_plot2['parameters.reg_spikes']),
        dict(tickvals=np.arange(np.nanmin(df_plot['parameters.reg_neurons']),
                                np.nanmax(
            df_plot['parameters.reg_neurons'])*1.05,
            2e-6),
            tickformat=".0s",
            label='reg_neurons', values=df_plot2['parameters.reg_neurons']),
        dict(range=[10, 90],
             tickvals=np.arange(10, 90+1, 20),
             label='Accuracy (%)', values=df_plot2['default'])
    ]),
    labelfont=dict(size=dimension_label_size),
    tickfont=dict(size=dimension_ticks_size)
)
)
fig.show()
